<a href="https://colab.research.google.com/github/cjchaitali/ADR_extraction/blob/main/Topic_modeling(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk; 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
pip install pyLDAvis

In [ ]:
pip install gensim

In [ ]:
pip install spacy==2.2.2

     |████████████████████████████████| 10.3MB 11.4MB/s 
     |████████████████████████████████| 2.2MB 25.9MB/s 
  Found existing installation: thinc 7.1.1
    Uninstalling thinc-7.1.1:
      Successfully uninstalled thinc-7.1.1
  Found existing installation: spacy 2.2.0
    Uninstalling spacy-2.2.0:
      Successfully uninstalled spacy-2.2.0


In [ ]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [ ]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [ ]:
# Import Dataset
df = pd.read_csv('Mirena_IUD.csv', encoding='cp1252',error_bad_lines=False)
df.head()

,Title,Time,reporttype,Reviews,Unnamed: 4
0,Rated for Birth Control Report,Posted 6 months ago (8/19/2020),Report,I got the Mirena in Feb 2020. The insertion wa...,NaN
1,Rated for Birth Control Report,Posted 6 months ago (8/19/2020),Report,This is 2nd time I have had the Mirena. Total ...,NaN
2,Rated for Birth Control Report,Posted 6 months ago (8/19/2020),Report,The biggest joke known to mankind. Doctor gave...,NaN
3,Rated for Birth Control Report,Posted 6 months ago (8/14/2020),Report,I’ve had mirena for exactly one month now and ...,NaN
4,Rated for Endometriosis Report,Posted 7 months ago (8/6/2020),Report,"I have a history of gushing, painful periods w...",NaN


In [ ]:
df.drop(columns = ['Title','Time','reporttype','Unnamed: 4'],inplace = True)
df.head()

,Reviews
0,I got the Mirena in Feb 2020. The insertion wa...
1,This is 2nd time I have had the Mirena. Total ...
2,The biggest joke known to mankind. Doctor gave...
3,I’ve had mirena for exactly one month now and ...
4,"I have a history of gushing, painful periods w..."


In [ ]:
# Convert to list
data = df.Reviews.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'"," ", sent) for sent in data]

pprint(data[:2])

['I got the Mirena in Feb 2020. The insertion wasn t pleasant by any means. It '
 'was like instant contractions and my hands were gripping the table. I was '
 'pretty uncomfortable for the rest of that evening and the next day but I was '
 'able to manage any discomfort with ibuprofen. For the next couple of months '
 'I just went about my life...my job changed, covid hit so I thought I was '
 'feeling out of control because of those things. Until I really started '
 'paying attention.....I noticed my emotions were all over the map. I couldn t '
 'keep myself under control, I was crying all of the time. Some days my '
 'boyfriend was looking at me like....what is wrong????? Then I noticed the '
 'severe change in my appetite, I was hungry ALL OF THE TIME. All I wanted was '
 'salt and sugar ALL DAY LONG. My skin was a mess, broken out all of the time '
 'on my face, chest and back. I was having no period, just like they said I '
 'would. BUT I was having PMS everyday. 24 hours a day, 

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence).encode('utf-8'), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))
print(data_words[:1])


[['got', 'the', 'mirena', 'in', 'feb', 'the', 'insertion', 'wasn', 'pleasant', 'by', 'any', 'means', 'it', 'was', 'like', 'instant', 'contractions', 'and', 'my', 'hands', 'were', 'gripping', 'the', 'table', 'was', 'pretty', 'uncomfortable', 'for', 'the', 'rest', 'of', 'that', 'evening', 'and', 'the', 'next', 'day', 'but', 'was', 'able', 'to', 'manage', 'any', 'discomfort', 'with', 'ibuprofen', 'for', 'the', 'next', 'couple', 'of', 'months', 'just', 'went', 'about', 'my', 'life', 'my', 'job', 'changed', 'covid', 'hit', 'so', 'thought', 'was', 'feeling', 'out', 'of', 'control', 'because', 'of', 'those', 'things', 'until', 'really', 'started', 'paying', 'attention', 'noticed', 'my', 'emotions', 'were', 'all', 'over', 'the', 'map', 'couldn', 'keep', 'myself', 'under', 'control', 'was', 'crying', 'all', 'of', 'the', 'time', 'some', 'days', 'my', 'boyfriend', 'was', 'looking', 'at', 'me', 'like', 'what', 'is', 'wrong', 'then', 'noticed', 'the', 'severe', 'change', 'in', 'my', 'appetite', 'wa

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(bigram_mod[bigram_mod[data_words[801]]])

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['chills', 'itching', 'mood_swings', 'abnormal', 'bleeding', 'cramps', 'nauseous', 'breasts', 'are', 'very', 'tender', 'and', 'back', 'pain']


In [ ]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)


In [ ]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [ ]:
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])

[['get', 'pleasant', 'mean', 'instant', 'contraction', 'hand', 'grip', 'table', 'pretty', 'uncomfortable', 'rest', 'evening', 'next', 'day', 'able', 'manage', 'discomfort', 'ibuprofen', 'next', 'couple', 'month', 'go', 'life', 'job', 'change', 'hit', 'think', 'feel', 'control', 'thing', 'really', 'start', 'pay', 'attention', 'notice', 'emotion', 'map', 'keep', 'control', 'cry', 'time', 'day', 'boyfriend', 'look', 'wrong', 'notice', 'severe', 'change', 'appetite', 'hungry', 'time', 'want', 'salt', 'sugar', 'day', 'long', 'skin', 'mess', 'break', 'time', 'face', 'chest', 'back', 'period', 'say', 'would', 'pms', 'everyday', 'hour', 'day', 'day', 'week', 'period', 'show', 'think', 'mess', 'mentally', 'body', 'want', 'period', 'bad', 'happen', 'slowly', 'first']]


In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1]) #term documnet matrix considering dictionary of words
#word 0 appeared 1 time in 1st document

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 2), (12, 1), (13, 1), (14, 5), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 2), (42, 1), (43, 2), (44, 2), (45, 1), (46, 3), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 2), (63, 3), (64, 1), (65, 2), (66, 1), (67, 1), (68, 1)]]


In [ ]:
id2word[2]

'attention'

In [ ]:
corpus[:1][0][:10] #here word id 8 appeared 2 times in 1st document

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 2),
 (9, 1)]

In [ ]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('able', 1),
  ('appetite', 1),
  ('attention', 1),
  ('back', 1),
  ('bad', 1),
  ('body', 1),
  ('boyfriend', 1),
  ('break', 1),
  ('change', 2),
  ('chest', 1),
  ('contraction', 1),
  ('control', 2),
  ('couple', 1),
  ('cry', 1),
  ('day', 5),
  ('discomfort', 1),
  ('emotion', 1),
  ('evening', 1),
  ('everyday', 1),
  ('face', 1),
  ('feel', 1),
  ('first', 1),
  ('get', 1),
  ('go', 1),
  ('grip', 1),
  ('hand', 1),
  ('happen', 1),
  ('hit', 1),
  ('hour', 1),
  ('hungry', 1),
  ('ibuprofen', 1),
  ('instant', 1),
  ('job', 1),
  ('keep', 1),
  ('life', 1),
  ('long', 1),
  ('look', 1),
  ('manage', 1),
  ('map', 1),
  ('mean', 1),
  ('mentally', 1),
  ('mess', 2),
  ('month', 1),
  ('next', 2),
  ('notice', 2),
  ('pay', 1),
  ('period', 3),
  ('pleasant', 1),
  ('pms', 1),
  ('pretty', 1),
  ('really', 1),
  ('rest', 1),
  ('salt', 1),
  ('say', 1),
  ('severe', 1),
  ('show', 1),
  ('skin', 1),
  ('slowly', 1),
  ('start', 1),
  ('sugar', 1),
  ('table', 1),
  ('thing', 

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.059*"need" + 0.048*"bleeding" + 0.036*"yesterday" + 0.033*"bloat" + '
  '0.025*"actually" + 0.021*"pelvic" + 0.020*"literally" + 0.019*"ultrasound" '
  '+ 0.019*"warn" + 0.018*"discharge"'),
 (1,
  '0.086*"feel" + 0.054*"pain" + 0.035*"get" + 0.030*"ever" + 0.026*"doctor" + '
  '0.025*"bad" + 0.025*"even" + 0.025*"go" + 0.024*"tell" + 0.023*"thing"'),
 (2,
  '0.057*"get" + 0.037*"bad" + 0.034*"week" + 0.033*"say" + 0.033*"would" + '
  '0.026*"try" + 0.026*"doctor" + 0.022*"never" + 0.022*"also" + '
  '0.021*"still"'),
 (3,
  '0.022*"choice" + 0.021*"product" + 0.019*"case" + 0.016*"fun" + 0.015*"son" '
  '+ 0.014*"convince" + 0.013*"fear" + 0.013*"sick" + 0.013*"mirror" + '
  '0.012*"obviously"'),
 (4,
  '0.079*"maybe" + 0.068*"regular" + 0.040*"other" + 0.036*"research" + '
  '0.026*"close" + 0.026*"fatigue" + 0.023*"wish" + 0.021*"suggest" + '
  '0.020*"size" + 0.019*"turn"'),
 (5,
  '0.059*"headache" + 0.048*"level" + 0.042*"thought" + 0.035*"inserted" + '
  '0.028*"concer

In [ ]:
# Perplexity Score
print('\nPerplexity Score: ', lda_model.log_perplexity(corpus)) 

# Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.775518041984831

Coherence Score:  0.4867293255905139


In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
14     0.333574 -0.009018       1        1  27.000238
1      0.301555 -0.070416       2        1  14.534797
6      0.265037  0.195773       3        1  12.392957
2      0.298270 -0.074820       4        1  10.838119
13     0.247928  0.194903       5        1   9.312452
10     0.144742 -0.296418       6        1   6.102280
3     -0.109383  0.041941       7        1   2.835959
8     -0.042867 -0.071606       8        1   2.734327
0     -0.053805  0.022279       9        1   2.212897
11    -0.094669  0.018960      10        1   1.948183
4     -0.113382 -0.007896      11        1   1.453569
17    -0.122321  0.024363      12        1   1.168032
19    -0.122268  0.005037      13        1   1.128780
7     -0.134843 -0.001255      14        1   1.081637
18    -0.121123  0.020623      15        1   1.037749
5     -0.126704 -0.012005      16        1   1.015989
9     -0.128256 -0.006219      17        1   0.941638
15    -0.131206  0.015515      18        1   0.920173
12    -0.139543  0.005242      19        1   0.820517
16    -0.150734  0.005018      20        1   0.519706, topic_info=           Term         Freq        Total Category  logprob  loglift
20         feel   718.000000   718.000000  Default  30.0000  30.0000
118   insertion   539.000000   539.000000  Default  29.0000  29.0000
150       cramp   760.000000   760.000000  Default  28.0000  28.0000
75         year   709.000000   709.000000  Default  27.0000  27.0000
46       period  1002.000000  1002.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
3528     freeze     1.156460     1.956145  Topic20  -5.4319   4.7340
1401  condition     3.048407     8.006881  Topic20  -4.4627   4.2940
397      forget     4.512700    18.818776  Topic20  -4.0704   3.8317
498        wear     2.604601    23.142696  Topic20  -4.6200   3.0753
657    probably     1.466733    23.195560  Topic20  -5.1943   2.4987

[861 rows x 6 columns], token_table=      Topic      Freq             Term
term                                  
1911     12  0.940722          abdoman
0         1  0.982412             able
617       9  0.957217         abnormal
1159      7  0.915088         abortion
964      18  0.720510         absolute
...     ...       ...              ...
75       10  0.001409             year
460       6  0.945290  yeast_infection
189       9  0.992512        yesterday
890       3  0.389697              yet
890       9  0.562895              yet

[1135 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[15, 2, 7, 3, 14, 11, 4, 9, 1, 12, 5, 18, 20, 8, 19, 6, 10, 16, 13, 17])